In [1]:
import os
from IPython.display import display, HTML

import numpy as np
import pandas as pd

# Investigating claims on performance made in the paper
LLMs perform worse on the proposed TempTabQA dataset than humans. The authors split data into a head and tail set, i.e., into more common and less common domains. Of all LLMs, GPT performs the best. Transforming tables to knowledge graphs improves performance further.

Key claims I want to investigate are:
- LLMs perform better in head set than in tail set. Human performance roughly similar.
- _How many_ questions are the hardest for humans. _Who_ questions are the hardest for LLMs
- LLMs and humans struggle with _summation_, _difference_, and _comparison_ questions.
- Explicit time-related questions are harder then implicit questions.
- Money, and person-related tail questions are hard for LLMs and humans.

## Inspect training, head, and tail datasets
In the following, I will start with inspecting the training data. Later, I look into the head and tail dataset where the head set is supposed to cover more common domains and where the tail data set is supposed to cover less common domains.

### Training data

In [2]:
df_train = pd.read_csv("../../data/maindata/qapairs/train-set/train-set.csv", index_col=0)

In [3]:
df_train.head()

,question,answer,category,table_id
0,How many years prior to Davis's death did she ...,57,actor,4
1,How much longer was Davis married to Gary Merr...,7 years,actor,4
2,How long after Davis career began did she take...,6,actor,4
3,Who was Bette Davis married to in 1942?,Arthur Farnsworth,actor,4
4,When was the first time that Bette Davis becam...,1943,actor,4


In [4]:
df_train.loc[:, "category"].unique()

array(['actor', 'swimming', 'wrestling', 'empire', 'legislature',
       'footballer', 'tennis', 'golf', 'rugby', 'nba', 'volleyball',
       'emperor', 'badminton', 'tabletennis', 'album', 'civil war',
       'conference', 'aircraft', 'economy', 'court', 'racing',
       'car driver', 'curling', 'nobel', 'painter', 'scientist', 'person',
       'body builder', 'christian leader', 'musician', 'baseball',
       'lacrosse', 'country', 'space probe', 'railway', 'football',
       'handball', 'athelete', 'martial artist', 'nfl', 'ice hockey',
       'basketball', 'company', 'website', 'university', 'stadium',
       'launchpad', 'sailor', 'book', 'skier', 'church', 'monument',
       'figure skating', 'office holder', 'song', 'concert', 'painting',
       'military conflict', 'war/conflict', 'current war', 'earthquake',
       'space program', 'music', 'character', 'movie', 'show',
       'rail line', 'agency', 'national cricket team', 'board game',
       'game', 'national football team'

In [5]:
df_train.shape

(7680, 4)

In [6]:
df_train.drop_duplicates(subset=["question", "answer"]).shape

(7299, 4)

In [7]:
def remove_persons_and_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop_duplicates(subset=["question", "answer"]).query(
        "category.isin(['empire', 'legislature', 'emperor', 'album', 'civil war', 'conference', 'aircraft','economy', 'court', 'nobel', 'country', 'space probe', 'railway', 'company','website', 'university', 'stadium', 'launchpad',  'book', 'church', 'monument', 'office holder','song', 'concert', 'painting', 'military conflict', 'war/conflict','current war', 'earthquake', 'space program', 'music', 'character','movie', 'show', 'rail line', 'agency','board game', 'game'])"
    )

In [8]:
df_without_persons = df_train.pipe(remove_persons_and_duplicates)

In [9]:
df_without_persons.groupby("category").sample(1)

,question,answer,category,table_id
7213,How many total child agencies are there in all...,69,agency,1140
1190,What was the first year that pilots were able ...,1913,aircraft,225
6256,How many singles were released on the Death Ma...,3,album,950
7257,How many years were between when FASA was firs...,23,board game,1150
4467,In what year did Animal Farm get published?,1945,book,701
6982,What year did Lassie release on CBS?,1954,character,1076
4594,What is the Length of All Saints’ Church?,120 feet (37 m),church,722
6512,Who was the leader of UK after 2 years of Mala...,Clement Attlee,civil war,996
3877,In what year was DollarTree founded?,1986,company,602
6815,When did the Beautiful Trauma World Tour start?,"March 1, 2018",concert,1053


### Head test set

In [12]:
df_head = pd.read_csv("../../data/maindata/qapairs/head-set/head-set.csv", index_col=0)
df_head.head()

,question,answer,category,table_id
0,How many years did Art Carney as actor since 1...,54 Years,actor,2
1,Who was the spouse of Art Carney in 1970?,Barbara Isaac,actor,2
2,How many total years was Art Carney married to...,28 years,actor,2
3,Which spouse was Art Carney married to the least?,Barbara Isaac,actor,2
4,Who was Art Carney married to when he served i...,Jean Myers,actor,2


In [13]:
df_head.shape

(1851, 4)

In [14]:
df_head_without_persons = df_head.pipe(remove_persons_and_duplicates)

In [15]:
df_head_without_persons.groupby("category").sample(1)

,question,answer,category,table_id
1719,Which year formed in Coordination Center for E...,1937,agency,1139
243,How many years did B-24 Skua was in active ser...,3 Years,aircraft,226
1390,How many labels issued Confessions of a Danger...,two,album,948
1784,When was the first edition of Atlantis released?,1984,board game,1163
1351,How long was Dick King-Smith married to his s...,10 years,book,929
1661,What was the last year the Monks was portrayed...,2007,character,1104
1108,When Valen kapell was closed?,1873-1949,church,715
1413,How many years did Libya supported to Ethiopia...,8 Years,civil war,979
893,Is there more or less employess at AT&T in 202...,less,company,597
1526,How many shows was showed in Europe and Austra...,29,concert,1054


In [16]:
diff_cat_train_head = np.setdiff1d(
    df_train.loc[:, "category"].unique(), df_head.loc[:, "category"].unique()
)
diff_cat_train_head

array(['conference', 'football', 'painting', 'war/conflict'], dtype=object)

In [17]:
diff_cat_head_train = np.setdiff1d(
    df_head.loc[:, "category"].unique(), df_train.loc[:, "category"].unique()
)
diff_cat_head_train

array(['event', 'fighter', 'navy vessel', 'politician'], dtype=object)

### Tail test set

In [19]:
df_tail = pd.read_csv("../../data/maindata/qapairs/tail-set/tail-set.csv", index_col=0)
df_tail.head()

,question,answer,category,table_id
0,What was the age when Jenson Button best finis...,Age of 38,f1,0
1,What was Jenson Button's age when he won the S...,38,f1,0
2,How long did it take Jenson Button to win his ...,9 years,f1,0
3,Where did Jenson Button compete 5 years before...,Brazil,f1,0
4,For how many years did Jenson Button's profess...,17 years,f1,0


In [20]:
df_tail.category.unique()

array(['f1', 'time zone', 'hockey', 'cricket', 'cycling', 'squash',
       'sumo', 'political party', 'sports event', 'holiday', 'ship',
       'orbitor', 'boxing', 'terrorist orgnization', 'cyclone',
       'proxy war', 'army', 'planet', 'disease'], dtype=object)

In [21]:
df_tail.groupby("category").sample(1)

,question,answer,category,table_id
959,How many years did First Lebanon War was held ...,3 Years,army,1119
552,How much longer did Yamamoto play for Kamatama...,3 years,boxing,484
615,How old was Brian Lara when he joined West Ind...,21,cricket,528
101,What team did Valverde ride for the year he wo...,Kelme–Costa Blanca,cycling,99
740,How strong was Hurricane Ivan?,Category 5 Hurricane (SSHWS),cyclone,958
1030,How much time elapsed between the first report...,14 days.,disease,1205
34,How many years are between Barrichello's best ...,2,f1,51
661,How old was Lombi when won gold at the Pan Ame...,24,hockey,680
436,How many days later does Mardi Gras start in 2...,5 days,holiday,218
547,How many month left did ExoMars was planned in...,9 months left,orbitor,461


In [22]:
df_train.query("category.isin(@df_tail.category.unique())")

,question,answer,category,table_id


In [23]:
df_head.query("category.isin(@df_tail.category.unique())")

,question,answer,category,table_id


In [24]:
df_tail = df_tail.drop_duplicates()
# Table contains duplicate questions. Either, because answers are slightly different or because category is different
df_tail[df_tail.loc[:, "question"].duplicated(keep=False)]

,question,answer,category,table_id
198,How old was Tarek Momen when he turned pro?,17,squash,110
202,How old was Tarek Momen when he turned pro?,17 years old,squash,110
219,How old was Futabayama Sadaji when he retired?,33 years old,sumo,134
222,How old was Futabayama Sadaji when he retired?,33,sumo,134
270,How old was Aminishiki Ryūji when he made his ...,19 years old,sumo,180
275,How old was Aminishiki Ryūji when he made his ...,19,sumo,180
279,How many total years (terms) does Joe Biden ho...,6 years (2 terms),political party,182
280,What positions did Joe Biden hold in 2008?,"The United States Senator from Delaware, Chair...",political party,182
287,Who was Joe Biden married to when he was first...,Unmarried,political party,182
293,What positions did Joe Biden hold in 2008?,"The United States Senator from Delaware, Chair...",political party,183


In [25]:
# Duplicates due to 1:n relationship between question and category, and question and table_id
df_tail[
    df_tail.duplicated(subset=["question", "answer", "category"], keep=False)
].sort_values(by="question")

,question,answer,category,table_id
937,At what age does Joe Biden join the democratic...,27,army,1118
923,At what age does Joe Biden join the democratic...,27,army,1116
862,At what age does Joe Biden join the democratic...,27,cyclone,1091
732,At what age does Joe Biden join the democratic...,27,cyclone,958
279,How many total years (terms) does Joe Biden ho...,6 years (2 terms),political party,182
297,How many total years (terms) does Joe Biden ho...,6 years (2 terms),political party,184
965,How many years after the second marriage did J...,44 years,army,1120
957,How many years after the second marriage did J...,44 years,army,1119
947,How many years after the second marriage did J...,44 years,army,1118
293,What positions did Joe Biden hold in 2008?,"The United States Senator from Delaware, Chair...",political party,183


In [26]:
# Duplicates due to different answers per question
df_tail[df_tail.duplicated(subset=["question", "table_id"], keep=False)].sort_values(
    by="question"
)

,question,answer,category,table_id
270,How old was Aminishiki Ryūji when he made his ...,19 years old,sumo,180
275,How old was Aminishiki Ryūji when he made his ...,19,sumo,180
219,How old was Futabayama Sadaji when he retired?,33 years old,sumo,134
222,How old was Futabayama Sadaji when he retired?,33,sumo,134
198,How old was Tarek Momen when he turned pro?,17,squash,110
202,How old was Tarek Momen when he turned pro?,17 years old,squash,110
850,Who was the US senator from Delaware in Dec 1972?,A. J. Caleb Boggs,cyclone,1091
854,Who was the US senator from Delaware in Dec 1972?,J. Caleb Boggs,cyclone,1091


## Inspect the author's data analysis
After running the dataAnalysis.py script, I created a .csv that contains descriptors of the questions raised in the training set such as whether the questions is implicit or explicit, which temporal operation is required and so on.

In [27]:
df_analysis = pd.read_csv(
    "../../data/maindata/qapairs/train-set/train-set_analysis.csv"
).iloc[:, 2:]
df_analysis.head()

,question,answer,category,table_id,explicit,implicit,before_related,after_related,duration_related,max_operator,min_operator,count_operator,sum_operator,difference_operator,average_operator,comparison_operator,ordinal,answer_type
0,How many years prior to Davis's death did she ...,57,actor,4,True,False,True,False,False,False,False,True,False,False,False,False,False,COUNT
1,How much longer was Davis married to Gary Merr...,7 years,actor,4,False,True,False,False,True,False,False,False,False,False,False,False,False,TEMPORAL
2,How long after Davis career began did she take...,6,actor,4,False,True,False,True,False,False,False,False,False,False,False,False,False,UNKNOWN
3,Who was Bette Davis married to in 1942?,Arthur Farnsworth,actor,4,True,False,False,False,False,False,False,False,False,False,False,False,False,PERSON
4,When was the first time that Bette Davis becam...,1943,actor,4,False,True,False,False,False,False,False,False,False,False,False,False,True,TEMPORAL


In [28]:
df_analysis.columns

Index(['question', 'answer', 'category', 'table_id', 'explicit', 'implicit',
       'before_related', 'after_related', 'duration_related', 'max_operator',
       'min_operator', 'count_operator', 'sum_operator', 'difference_operator',
       'average_operator', 'comparison_operator', 'ordinal', 'answer_type'],
      dtype='object')

In [29]:
df_analysis.loc[:, "answer_type"].unique()

array(['COUNT', 'TEMPORAL', 'UNKNOWN', 'PERSON', 'AGE', 'PLACE',
       'PERCENTAGE', 'ORDINAL', 'YES/NO', 'ORGANIZATION', 'PRODUCT',
       'MONEY'], dtype=object)

### _How many_ questions are the hardest for humans. _Who_ questions are the hardest for LLMs
Let's investigate how these questions look like and what they have in common.

`dataAnalysis.py` does not produce a category _how many_ or _who_. When investigating the regexes in the script, both categories `count_operator` and `duration_related` match _how many_. When looking for _who_ matching regexes, only answer types were classified. Therefore, I assume the analysis must have been using simple string matching.

#### Let's start with looking at the GPT 4 prediction for the tail data set. 
We must load both datasets, remove duplicates and find tail questions by merging them with the GPT4 answers. Afterwards, we will search for _how many_ and _who_ questions to inspect the data further.

In [30]:
df_gpt4 = pd.read_csv(
    "../../models/predictions/gpt4/fewshot_with_reasoning/outdomain_eval_gpt_4_few_shot_with_reasoning_single.csv"
)
df_gpt4.head()

,table,predicted_answer,actual_answer,question
0,"<html><body><table class=""infobox biography vc...",Formula One && Five years before his Super GT ...,Brazil,Where did Jenson Button compete 5 years before...
1,"<html><body><table class=""infobox biography vc...",20 years old && Button began his Formula One c...,20,How old was Jenson Button when he began his pr...
2,"<html><body><table class=""infobox biography vc...",United Kingdom && Jenson Button's nationality ...,England,Which country did Jenson Button participated i...
3,"<html><body><table class=""infobox biography vc...",2000 && He first entered the Australian Grand ...,2000,When did Jenson Button first enter into the Au...
4,"<html><body><table class=""infobox biography vc...",10 years ago && Jenson Button last won the Bra...,10 Years ago (2012),How many years ago did Jenson Button last won ...


In [31]:
def inline_render_html(html_str):
    return f'<div style="border: 1px solid #ddd; padding: 5px;">{html_str}</div>'


def html_render_df(df, html_col="table"):
    return df.style.format({html_col: inline_render_html}).set_properties(
        subset=["table"], **{"text-align": "left"}
    )


html_render_df(df_gpt4.tail(1))


In [32]:
df_gpt4.shape

(1027, 4)

In [33]:
# Answers are otherwise not easy to check for exact match
df_gpt4 = df_gpt4.assign(
    predicted_answer_short=lambda x: x["predicted_answer"]
    .str.split("&&")
    .str[0]
    .str.strip()
)

In [34]:
html_render_df(
    df_gpt4.query("predicted_answer_short == actual_answer")
    .sample(3, random_state=94)
    .loc[:, ["table", "question", "predicted_answer_short", "actual_answer"]]
)

In [35]:
html_render_df(
    df_gpt4.query("predicted_answer_short == actual_answer")
    .sample(20, random_state=94)
    .loc[:, ["table", "question", "predicted_answer_short", "actual_answer"]]
)

In [36]:
html_render_df(
    df_gpt4.query("predicted_answer_short != actual_answer")
    .query("question.str.contains('Biden')")
    .loc[[279]]
)

In [37]:
df_gpt4.query("predicted_answer.str.contains('relevant')").shape

(15, 5)

In [38]:
html_render_df(
    df_gpt4.query("predicted_answer_short != actual_answer")
    .sample(10, random_state=94)
    .loc[:, ["table", "question", "predicted_answer_short", "actual_answer"]]
)

In [39]:
# Some questions appear more than one time
df_gpt4.groupby("question").size().to_frame("occurences_of_same_question").groupby(
    "occurences_of_same_question"
).size()

occurences_of_same_question
1    975
2      3
4      2
5      4
6      3
dtype: int64

In [40]:
df_gpt4[df_gpt4.duplicated(["question"], keep=False)].shape

(52, 5)

In [41]:
df_gpt4[df_gpt4.duplicated(["question", "actual_answer"], keep=False)].shape

(49, 5)

In [42]:
# Duplicate questions where question-actual_answer-pairs are not duplicated
(
    df_gpt4[df_gpt4.duplicated(["question"], keep=False)]  # Rows with duplicates only
    .drop_duplicates(
        subset=["question", "actual_answer"], keep=False
    )  # Remove (all) rows if Q and A are not unique
    .sort_values("question")
)

,table,predicted_answer,actual_answer,question,predicted_answer_short
468,"<html><body><table class=""infobox"" style=""widt...",26 years && HMS Somerset was commissioned in 1...,28 Years,How many years does HMSAstute is service in UK...,26 years
893,"<table class=""infobox"" style=""width:25.5em;bor...",8 years && HMS Astute has been in service sinc...,8 Years (from 2014),How many years does HMSAstute is service in UK...,8 years
938,"<table class=""infobox"" style=""width:25.5em;bor...",Not applicable && The context given is about t...,A. J. Caleb Boggs,Who was the US senator from Delaware in Dec 1972?,Not applicable


In [43]:
# Different predicted answers for the same question-table pairs
df_gpt4[df_gpt4.duplicated(["question", "table"], keep=False)]

,table,predicted_answer,actual_answer,question,predicted_answer_short
18,"<html><body><table class=""infobox ib-time-zone...",2 hours && Florida is in Eastern Time Zone and...,two hours,What's the difference in time between Florida ...,2 hours
20,"<html><body><table class=""infobox ib-time-zone...",2 hours difference && Florida is in Eastern Ti...,two hours,What's the difference in time between Florida ...,2 hours difference
219,"<html><body><table class=""infobox vcard""><tbod...",25 years old && Futabayama Sadaji was born in ...,25,How old was Futabayama Sadaji when he achieved...,25 years old
221,"<html><body><table class=""infobox vcard""><tbod...",25 years old && Futabayama was born in 1912 an...,25,How old was Futabayama Sadaji when he achieved...,25 years old


In [44]:
# Predicted answer and question are the same despite different tables
df_gpt4[df_gpt4.duplicated(["question", "predicted_answer"], keep=False)].sort_values(
    "question"
)

,table,predicted_answer,actual_answer,question,predicted_answer_short
342,"<html><body><table class=""infobox vevent""><tbo...",Neilia Hunter && Joe Biden was married to Neil...,Unmarried,Who was Joe Biden married to when he was first...,Neilia Hunter
728,"<table class=""infobox vevent"" style=""border-co...",Neilia Hunter && Joe Biden was married to Neil...,Unmarried,Who was Joe Biden married to when he was first...,Neilia Hunter
345,"<html><body><table class=""infobox vevent""><tbo...",Not relevant && The context provided does not ...,Neilia Hunter,Who was the better half of Joe Biden in 1970?,Not relevant
948,"<table class=""infobox"" style=""width:25.5em;bor...",Not relevant && The context provided does not ...,Neilia Hunter,Who was the better half of Joe Biden in 1970?,Not relevant


In [45]:
# Sanity check to see if table 342 and 345 are different
# (as prodced by this code: df_gpt4[df_gpt4.duplicated(["question", "predicted_answer"], keep=False)].sort_values("question"))
df_gpt4[df_gpt4.duplicated(["question", "predicted_answer", "table"], keep=False)]

,table,predicted_answer,actual_answer,question,predicted_answer_short


In [46]:
# Duplicates may be due to different predicted answers or different context tables
df_gpt4[
    df_gpt4.duplicated(subset=["question", "actual_answer"], keep=False)
].sort_values(by="question").head()

,table,predicted_answer,actual_answer,question,predicted_answer_short
908,"<table class=""infobox"" style=""width:25.5em;bor...",Not available && The context provided is about...,27,At what age does Joe Biden join the democratic...,Not available
725,"<table class=""infobox vevent"" style=""border-co...",Information not available && The context provi...,27,At what age does Joe Biden join the democratic...,Information not available
922,"<table class=""infobox"" style=""width:25.5em;bor...",Not available && The context provided does not...,27,At what age does Joe Biden join the democratic...,Not available
845,"<table class=""infobox vevent""><tbody><tr><th c...",Unrelated context && The provided context does...,27,At what age does Joe Biden join the democratic...,Unrelated context
339,"<html><body><table class=""infobox vevent""><tbo...",Not applicable && The context provided is not ...,one,How many parties does Joe’s Biden’s is associa...,Not applicable


In [49]:
html_tables = os.listdir("../../data/maindata/tables/html/")
table_id_to_html = {}
for file in html_tables:
    with open(f"../../data/maindata/tables/html/{file}") as f:
        html = f.read()
    table_id = int(file.replace(".html", ""))
    table_id_to_html[table_id] = html
table_id_mapping = pd.DataFrame.from_dict(
    table_id_to_html, orient="index", columns=["table"]
).reset_index(names="table_id")